<a href="https://colab.research.google.com/github/thePegasusai/CloudPhone/blob/main/Text_to_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2
from IPython import display
import math
import os

import tensorflow_hub as hub
import tensorflow.compat.v2 as tf

In [ ]:
model_address = 'https://tfhub.dev/deepmind/mil-nce/s3d/1'
load_model = hub.load(model_address)

def embedding_generator(model, input_sequence, input_word):

  video_output = model.signatures['video'](tf.constant(tf.cast(input_sequence, dtype= tf.float32)))
  text_output = model.signatures['text'](tf.constant(input_word))
  return video_output['video_embedding'],text_output['text_embedding']

In [ ]:
def center_crop_square(frame):
  y, x = frame.shape[0:2]
  dim_min = min(y, x)
  x_start = (x // 2) - (dim_min // 2)
  y_start = (y // 2) - (dim_min // 2)
  return frame[y_start:y_start + dim_min, x_start:x_start + dim_min] 

def video_load(video_url, max_frames = 40, resize = (224,224)):
  path_ = tf.keras.utils.get_file(os.path.basename(video_url)[-128:], video_url)
  cap = cv2.VideoCapture(path_)
  frames = []

  try:
    while True:
      rett, frame = cap.read()
      if not rett:
        break
      frame = center_crop_square(frame)
      frame = cv2.resize(frame,resize)
      frame = frame[:, :, [2, 1, 0]]
      frames.append(frame)

      if len(frames) == max_frames:
        break
  finally:
    cap.release()
  frames = np.array(frames)
  if len(frames) < max_frames:
    n_repeat = int(math.ceil(max_frames / float(len(frames))))
    frames = frames.repeat(n_repeat, axis = 0)
  frames = frames[:max_frames]
  return frames / 255.0

def video_display(urls):
    html = '<table>'
    html += '<tr><th>Video 1</th><th>Video 2</th><th>Video 3</th></tr><tr>'
    for url in urls:
        html += '<td>'
        html += '<img src="{}" height="224">'.format(url)
        html += '</td>'
    html += '</tr></table>'
    return display.HTML(html)  

def display_result(query, urls, scores):
  sorted_ix = np.argsort(-scores)
  html = ''
  html += '<h2>Input query: <i>{}</i> </h2><div>'.format(query)
  html += 'Results: <div>'
  html += '<table>'
  html += '<tr><th>Rank #1, Score:{:.2f}</th>'.format(scores[sorted_ix[0]])
  html += '<th>Rank #2, Score:{:.2f}</th>'.format(scores[sorted_ix[1]])
  html += '<th>Rank #3, Score:{:.2f}</th></tr><tr>'.format(scores[sorted_ix[2]])
  for i, idx in enumerate(sorted_ix):
    url = urls[sorted_ix[i]];
    html += '<td>'
    html += '<img src="{}" height="224">'.format(url)
    html += '</td>'
  html += '</tr></table>'
  return html

In [ ]:
video_1_url = 'https://upload.wikimedia.org/wikipedia/commons/b/b0/YosriAirTerjun.gif'
video_2_url = 'https://upload.wikimedia.org/wikipedia/commons/e/e6/Guitar_solo_gif.gif'
video_3_url = 'https://upload.wikimedia.org/wikipedia/commons/3/30/2009-08-16-autodrift-by-RalfR-gif-by-wau.gif'

video_1 = video_load(video_1_url)
video_2 = video_load(video_2_url)
video_3 = video_load(video_3_url)
all_videos = [video_1, video_2, video_3]

query_1_video = 'waterfall'
query_2_video = 'playing guitar'
query_3_video = 'car drifting'
all_queries_video = [query_1_video, query_2_video, query_3_video]
all_videos_urls = [video_1_url, video_2_url, video_3_url]
video_display(all_videos_urls)

Video 1,Video 2,Video 3
,,


In [ ]:
# video input
videos_ = np.stack(all_videos, axis = 0)

# prepare text input
words_ = np.array(all_queries_video)

# generate video and text embedding
video_embd, text_emdb = embedding_generator(load_model,videos_,words_)

#scores between video and text
all_scores = np.dot(text_emdb,tf.transpose(video_embd))

In [ ]:
all_scores

array([[ 4.5252433, -3.6661289, -1.1018997],
       [-1.66241  ,  6.456847 , -2.249011 ],
       [-0.7754799, -1.7600968,  8.881555 ]], dtype=float32)

In [ ]:
html = ''
for i, words in enumerate(words_):
  html += display_result(words, all_videos_urls, all_scores[i, :])
  html += '<br>'
display.HTML(html)

"Rank #1, Score:4.53","Rank #2, Score:-1.10","Rank #3, Score:-3.67"
,,
"Rank #1, Score:6.46","Rank #2, Score:-1.66","Rank #3, Score:-2.25"
,,
"Rank #1, Score:8.88","Rank #2, Score:-0.78","Rank #3, Score:-1.76"
,,
